In [1]:
import pickle
import anndata2ri
import anndata
import scanpy
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

#This activates the object conversion.
anndata2ri.activate()
%load_ext rpy2.ipython

/tmp/ipykernel_2813657/2190637113.py:10: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


In [2]:
#anndata.settings.allow_write_nullable_strings = True

In [3]:
#Import needed R libraries.
seurat = importr('Seurat')
base = importr('base')


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

In [4]:
#Read Seurat object into R.
robjects.r('seurat_object = readRDS("/scratch/hnatri/PIPAC/cell_merged_spatial_filtered_splitsamples.rds")')

<rpy2.robjects.methods.RS4 object at 0x1555515ddfc0> [RTYPES.S4SXP]
R classes: ('Seurat',)

In [5]:
#Merge layers because Seurat object versioning is wild. Convert Seurat object to Anndata.
#robjects.r('seurat_object = JoinLayers(seurat_object)')
adata = robjects.r('as.SingleCellExperiment(seurat_object)')

/opt/conda/envs/rpy2/lib/python3.10/site-packages/anndata2ri/_r2py.py:132: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  return AnnData(exprs, obs, var, uns, obsm, layers=layers)


In [6]:
adata.layers

Layers with keys: logcounts

In [7]:
#Check to make sure conversion worked and possibly cast counts to array if necessary.
adata.X = adata.X.toarray()

In [8]:
adata.X

array([[0., 0., 2., ..., 0., 0., 0.],
       [0., 0., 0., ..., 2., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [9]:
del adata.layers

In [10]:
#Write anndata to pickle file. Next move to the clustering script.
with open("/scratch/hnatri/PIPAC/cell_merged_spatial_filtered_splitsamples", "wb") as adata_object:
    pickle.dump(adata, adata_object)

In [11]:
adata

AnnData object with n_obs × n_vars = 1161604 × 480
    obs: 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'genomic_control_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'nucleus_count', 'segmentation_method', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'num.blank', 'TMA', 'percent.blank', 'nCount_cell_RNA', 'nFeature_cell_RNA', 'Sample', 'ident'
    obsm: 'SP'

In [12]:
adata.obs["nFeature_RNA"] = adata.obs["nFeature_RNA"].astype('float64')
adata.obs["transcript_counts"] = adata.obs["transcript_counts"].astype('float64')
adata.obs["control_probe_counts"] = adata.obs["control_probe_counts"].astype('float64')
adata.obs["genomic_control_counts"] = adata.obs["genomic_control_counts"].astype('float64')
adata.obs["unassigned_codeword_counts"] = adata.obs["unassigned_codeword_counts"].astype('float64')
adata.obs["deprecated_codeword_counts"] = adata.obs["deprecated_codeword_counts"].astype('float64')
adata.obs["total_counts"] = adata.obs["total_counts"].astype('float64')
adata.obs["nucleus_count"] = adata.obs["nucleus_count"].astype('float64')
adata.obs["nFeature_cell_RNA"] = adata.obs["nFeature_cell_RNA"].astype('float64')

In [13]:
adata.obs.dtypes

orig.ident                      object
nCount_RNA                     float64
nFeature_RNA                   float64
cell_id                         object
x_centroid                     float64
y_centroid                     float64
transcript_counts              float64
control_probe_counts           float64
genomic_control_counts         float64
control_codeword_counts          Int32
unassigned_codeword_counts     float64
deprecated_codeword_counts     float64
total_counts                   float64
cell_area                      float64
nucleus_area                   float64
nucleus_count                  float64
segmentation_method             object
num.blank                      float64
TMA                           category
percent.blank                  float64
nCount_cell_RNA                float64
nFeature_cell_RNA              float64
ident                         category
dtype: object

In [27]:
adata.write_h5ad(filename="/scratch/hnatri/PIPAC/cell_merged_spatial_filtered_splitsamples.h5ad")

In [28]:
#adata.obs["ncells3k_niche_k8_n20"].unique()